# BA2Plus Team Submission

### 전체 프로세스 개요:

- 자체제작한, 별도의 크롤링 패키지로 KRX/Naver 에서 가격, 거개량, 유동성, 시총 등의 데이터를 가져옴. 
    - 코드: https://github.com/jaepil-choi/korquanttools
    - 각각의 데이터를 `.pickle` 로 저장해 본 코드에서 써먹음 
    - pickle data를 받아올 수 있는 google drive 첨부: https://drive.google.com/drive/folders/1fR9fiogdhdktHuX_LbgBdMxfgHSkFT1y?usp=sharing
- 데이터를 불러와 public 기간 전까지 자르고, 데이터를 조합하여 새로운 변수를 만듦
    - 따라서 look-ahead 없음
    - 새로운 변수: `close t-1`, `close t-3`, `close t-5`, `normalized rdv/adv`
    - 각 세팅은 별도의 `submission_config.py` 모듈로 관리. (뒷부분 첨부)
- 모델에 넣고 돌림 
    - XGB + LGBM base model 
    - step 1, step 2, ... , step 15에 대해 따로 예측함 
    - 시그널 만듦 
- submission 형식에 맞게 변환
    - `submission_util.py` 모듈로 형식에 맞게 변환함. (뒷부분 첨부)

### 코드 실행환경 및 실행방법
- 코드 실행환경
    - python 3.9
    - xgboost, sklearn, lightgbm, tqdm, pandas, numpy, catboost, statsmodels 필요
    
- 실행방법 (중요)
    - .ipynb만 제출할 수 있다는 대회 제약 때문에 부득이 .py 모듈을 후반부에 첨부. 이 파일들이 있어야 코드가 돌아감. 
    - 코드를 실행하려면 drive 링크의 pickle 파일들을 받아 `/data` 폴더에 넣고, output을 넣을 `/output` 폴더도 만들어줘야 함.
    - 또한, `/data` 폴더 내에 `train.csv`, `train_additional.csv`파일을 넣어야 함.
    - 그리고 노트북과 같은 폴더 안에 `submission_config.py` 와 `submission_config.py`, `sample_submission.csv` 파일이 위치해야 함. 


나머지 과정은 아래 markdown 참고

## Basic settings

### Import libraries

In [ ]:
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor

from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.impute import SimpleImputer

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
## custom library

import eda_util as eutil
import submission_config as subconfig
import submission_util as subutil

In [ ]:
pd.set_option('display.float_format', lambda x: f'{x:,g}')

In [ ]:
BASE_PATH = subconfig.BASE_PATH
DATA_PATH = subconfig.DATA_PATH

OUTPUT_PATH = subconfig.OUTPUT_PATH

### Import data & preprocessing

In [ ]:
krx_df = pd.read_csv(subconfig.krx_df_PATH)

In [ ]:
krx_df.columns = ['date', 'code', 'name', 'volume', 'open', 'high', 'low', 'close']

In [ ]:
krx_df['date'] = pd.to_datetime(krx_df['date'], format='%Y%m%d')

In [ ]:
return_df = pd.read_pickle(subconfig.return_df_PATH)
close_df = pd.read_pickle(subconfig.adjclose_df_PATH)

In [ ]:
open_df = pd.read_pickle(subconfig.adjopen_df_PATH)
high_df = pd.read_pickle(subconfig.adjhigh_df_PATH)
low_df = pd.read_pickle(subconfig.adjlow_df_PATH)

In [ ]:
## date list

holidays = return_df.isnull().all(axis=1)
tradingdays = ~holidays

holidays = holidays.index[holidays]
tradingdays = tradingdays.index[tradingdays]

In [ ]:
TRAIN_START = pd.to_datetime(subconfig.TRAIN_START, format='%Y-%m-%d')
REALOS_PORTFOLIO_DATE = pd.to_datetime(subconfig.REALOS_PORTFOLIO_DATE, format='%Y-%m-%d')

In [ ]:
tradingdays = tradingdays[(tradingdays >= TRAIN_START) & (tradingdays <= REALOS_PORTFOLIO_DATE)]

In [ ]:
dacon_sid_list = [ii[1:] for ii in krx_df['code'].unique()]

In [ ]:
return_df = return_df.loc[tradingdays, :].dropna(axis='columns', how='all')
return_df = return_df.loc[:, dacon_sid_list]

close_df = close_df.loc[tradingdays, :].dropna(axis='columns', how='all')
close_df = close_df.loc[:, dacon_sid_list]

In [ ]:
open_df = open_df.loc[tradingdays, :].dropna(axis='columns', how='all')
open_df = open_df.loc[:, dacon_sid_list]

high_df = high_df.loc[tradingdays, :].dropna(axis='columns', how='all')
high_df = high_df.loc[:, dacon_sid_list]

low_df = low_df.loc[tradingdays, :].dropna(axis='columns', how='all')
low_df = low_df.loc[:, dacon_sid_list]

In [ ]:
# SIMOS_START = subconfig.SIMOS_START
# simOS_END = subconfig.SIMOS_END

### Import additional data

In [ ]:
volume_df = pd.read_pickle(subconfig.volume_df_PATH)
dollarvolume_df = pd.read_pickle(subconfig.dollarvolume_df_PATH)
marketcap_df = pd.read_pickle(subconfig.marketcap_df_PATH)
market_cat_df = pd.read_pickle(DATA_PATH / 'market_cat_df_20140101_20230730.pickle')

In [ ]:
volume_df = volume_df.loc[tradingdays, :].dropna(axis='columns', how='all')
volume_df = volume_df.loc[:, dacon_sid_list]

dollarvolume_df = dollarvolume_df.loc[tradingdays, :].dropna(axis='columns', how='all')
dollarvolume_df = dollarvolume_df.loc[:, dacon_sid_list]

marketcap_df = marketcap_df.loc[tradingdays, :].dropna(axis='columns', how='all')
marketcap_df = marketcap_df.loc[:, dacon_sid_list]

In [ ]:
market_cat_inrange = market_cat_df[market_cat_df['trdDd'].isin(tradingdays)]

In [ ]:
KOSPI_sid_list = market_cat_inrange[market_cat_inrange['is_KOSPI'] == True]['ISU_SRT_CD'].unique()
KOSDAQ_sid_list = market_cat_inrange[market_cat_inrange['is_KOSDAQ'] == True]['ISU_SRT_CD'].unique()
KONEX_sid_list = market_cat_inrange[market_cat_inrange['is_KONEX'] == True]['ISU_SRT_CD'].unique()

### Parameters

In [ ]:
REALOS_PORTFOLIO_DATE = subconfig.REALOS_PORTFOLIO_DATE

RDVADV_WINDOW = subconfig.WINDOWS['rdvadv'] # 20

### normalized RDV/ADV signal

In [ ]:
adv_df = dollarvolume_df.rolling(RDVADV_WINDOW, ).mean().dropna(axis='rows', how='all')

분모: average RDV/ADV ratio


In [ ]:
avg_adv_s = adv_df.mean(axis='columns')
avg_rdv_s = dollarvolume_df.iloc[RDVADV_WINDOW:, :].mean(axis='columns')

In [ ]:
avg_rdvadv_s = avg_rdv_s / avg_adv_s

분자: individual RDV/ADV ratio 

In [ ]:
ii_rdvadv_df = dollarvolume_df.iloc[RDVADV_WINDOW:, :] / adv_df

In [ ]:
normalized_rdvadv_signal_df = ii_rdvadv_df.divide(avg_rdvadv_s, axis='rows')
normalized_rdvadv_signal_df

In [ ]:
MODEL_TRAIN_START = pd.to_datetime('2021-06-29', format='%Y-%m-%d')

## Alphas

### Integrating my data with Insoo's code

In [ ]:
# Your function to calculate SMAPE
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [ ]:
results_df = pd.DataFrame(columns=['code'] + ['return_day_' + str(i) for i in range(1, 16)])

preds_df_fin_xgb = pd.DataFrame()
smapes_df_fin_xgb = pd.DataFrame()

preds_df_fin_lgbm = pd.DataFrame()
smapes_df_fin_lgbm = pd.DataFrame()

preds_df_fin_catboost = pd.DataFrame()
smapes_df_fin_catboost = pd.DataFrame()

아래 코드는 Ryzen 5 5600X 6 Core (CPU 12) 로 돌렸을 때 

40분 가량 걸림. 

Windows에서 GPU 연산은 활용하기 어려움. 

- XGB: conda는 지원안함, Windows는 version conflict 남
- LGBM: Linux만 지원

In [ ]:
# Iterate over each unique stock
for code in tqdm(dacon_sid_list):
    
    # Filter by stock code
    # Note: All prices are adjusted
    # TODO: Add normalized rdvadv signal to the columns

    train_close = pd.DataFrame(
        data={
            'open': open_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'high': high_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'low': low_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'close': close_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'close_t-1': close_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code].shift(1),
            'close_t-3': close_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code].shift(3),
            'close_t-5': close_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code].shift(5),
            'dollarvolume': dollarvolume_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'marketcap': marketcap_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'norm_rdvadv': normalized_rdvadv_signal_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
        }
        )
    train_close = train_close.iloc[5:, :] # nan 있는 1st row 제거 

    # Store original data for reference
    original_data = train_close.copy()

    # Create return columns for each day
    returns = []
    smapes_xgb = []
    smapes_lgbm = []
    smapes_catboost = []

    preds_df_xgb = pd.DataFrame()
    preds_df_lgbm = pd.DataFrame()
    preds_df_catboost = pd.DataFrame()
    
    # For each day from 1 to 15
    for day in range(1, 16):
        # Scale data
        X = train_close[:]
        y = train_close['close']
        
        scaler = MinMaxScaler(feature_range=(-1, 1))
        data_scaled = scaler.fit_transform(X)
        data_scaled2 = y
        
        X_train = data_scaled[:-day]
        y_train = data_scaled2[day:]
        X_test = data_scaled[-day]
        
        X_train = X_train[:int(len(X_train) * 0.9)]
        X_val = X_train[int(len(X_train) * 0.9):]
        y_train = y_train[:int(len(y_train) * 0.9)]
        y_val = y_train[int(len(y_train) * 0.9):] 

        # Train XGBoost
        xgb_model = XGBRegressor()
        xgb_model.fit(X_train, y_train)
        vals_xgb = xgb_model.predict(X_val)
        smapes_xgb.append(smape(y_val, vals_xgb))
        
        preds_xgb = xgb_model.predict([data_scaled[-day]])
        preds_df_xgb = pd.concat([preds_df_xgb, pd.DataFrame(preds_xgb)], axis = 0)

        # Train LightGBM
        lgbm_model = LGBMRegressor(verbose = -1)
        lgbm_model.fit(X_train, y_train)
        vals_lgbm = lgbm_model.predict(X_val)
        smapes_lgbm.append(smape(y_val, vals_lgbm))
        
        preds_lgbm = lgbm_model.predict([data_scaled[-day]])
        preds_df_lgbm = pd.concat([preds_df_lgbm, pd.DataFrame(preds_lgbm)], axis = 0)
    
    smapes_df_xgb = pd.DataFrame(smapes_xgb)
    smapes_df_lgbm = pd.DataFrame(smapes_lgbm)
    smapes_df_catboost = pd.DataFrame(smapes_catboost)

    preds_df_fin_xgb = pd.concat([preds_df_fin_xgb, preds_df_xgb], axis = 1)
    smapes_df_fin_xgb = pd.concat([smapes_df_fin_xgb, smapes_df_xgb], axis = 1)

    preds_df_fin_lgbm = pd.concat([preds_df_fin_lgbm, preds_df_lgbm], axis = 1)
    smapes_df_fin_lgbm = pd.concat([smapes_df_fin_lgbm, smapes_df_lgbm], axis = 1)

In [ ]:
smapes_df_fin_xgb.shape

In [ ]:
smapes_df_fin_lgbm.shape

In [ ]:
final = np.zeros((len(smapes_df_fin_xgb), len(smapes_df_fin_xgb.columns)))

for i in range(0, len(smapes_df_fin_xgb.columns)):
    for j in range(0, len(smapes_df_fin_xgb)):
        weights = [1 / smapes_df_fin_xgb.iloc[j:j+1, i].values[0],
                   
                   1 / smapes_df_fin_lgbm.iloc[j:j+1, i].values[0]]
        

        weights /= np.sum(weights) 
        

        final[j][i] = weights[0] * preds_df_fin_xgb.iloc[j:j+1, i].values[0] \
                            + weights[1] * preds_df_fin_lgbm.iloc[j:j+1, i].values[0] \
                        #     + weights[2] * preds_df_fin_lgbm.iloc[j:j+1, i].values[0]

In [ ]:
final_df = pd.DataFrame(final)
final_values = pd.DataFrame((final_df.iloc[-1] - final_df.iloc[0]) / final_df.iloc[0])
final_values_sharpe = -pd.DataFrame(((final_df.iloc[-1] - final_df.iloc[0]) / final_df.iloc[0]) / final_df.pct_change().std())

In [ ]:
final_values_sharpe.index = dacon_sid_list
final_values_sharpe.columns = ['VALUE']
final_values_sharpe.reset_index(inplace = True)
final_values_sharpe.columns = ['종목코드', 'VALUE']

In [ ]:
final_values_sharpe.set_index('종목코드', inplace = True)

In [ ]:
final_values_sharpe['VALUE']

### Submission

In [ ]:
alpha_feat_insoo = subutil.Submission(
    alpha_series=final_values_sharpe['VALUE'],
    alpha_name='alpha_feat_insoo_lagged_ReverseSharpe-final',
)

In [ ]:
alpha_feat_insoo.get_rank(export_path=OUTPUT_PATH)

## 함께 사용된 Python 모듈

`submission_config.py`

In [ ]:
from pathlib import Path

## Path configs

BASE_PATH = Path('.').resolve()
DATA_PATH = BASE_PATH / 'data'
OUTPUT_PATH = BASE_PATH / 'output'

krx_df_PATH = DATA_PATH / 'train.csv'
return_df_PATH = DATA_PATH / 'return_20140101_20230730.pkl'
adjclose_df_PATH = DATA_PATH / 'adjClose_20140101_20230730.pkl'
adjhigh_df_PATH = DATA_PATH / 'adjHigh_20140101_20230730.pkl'
adjlow_df_PATH = DATA_PATH / 'adjLow_20140101_20230730.pkl'
adjopen_df_PATH = DATA_PATH / 'adjOpen_20140101_20230730.pkl'
volume_df_PATH = DATA_PATH / 'volume_df_20140101_20230730.pkl'
dollarvolume_df_PATH = DATA_PATH / 'dollarvolume_df_20140101_20230730.pkl'
marketcap_df_PATH = DATA_PATH / 'marketcap_df_20140101_20230730.pkl'

## Param configs

# train (custom)
TRAIN_START = '2021-06-01'

# SimOS
PORTFOLIO_DATE = '2023-05-30' 
SIMOS_START = '2023-05-31'
SIMOS_END = '2023-06-21'

# RealOS
REALOS_PORTFOLIO_DATE = '2023-07-28' 
REALOS_START = '2023-07-31'

WINDOWS = {
    'rdvadv': 20,
}

`submission_util.py`

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    roc_auc_score, 
    roc_curve, 
    auc
    )
# TODO: SimOS 에서의 정답을 알고있다. 그러므로 eval metric 계산할 수 있다. 

import submission_config as subconfig

## Params
DACON_SID_CNT = 2000
SIMOS_START = subconfig.SIMOS_START
SIMOS_END = subconfig.SIMOS_END

## Import data
krx_df = pd.read_csv(subconfig.krx_df_PATH)
adjclose_df = pd.read_pickle(subconfig.adjclose_df_PATH)
return_df = pd.read_pickle(subconfig.return_df_PATH)

def get_simos_data(return_df, adjclose_df):
    holidays = return_df.isnull().all(axis=1)
    tradingdays = ~holidays

    holidays = holidays.index[holidays]
    tradingdays = tradingdays.index[tradingdays]

    return_df = return_df.loc[tradingdays, :]
    adjclose_df = adjclose_df.loc[tradingdays, :]

    return_df = return_df.loc[SIMOS_START:SIMOS_END, :]
    adjclose_df = adjclose_df.loc[SIMOS_START:SIMOS_END, :]

    return return_df, adjclose_df

# TODO: Confusing if global variables are not capitalized
simos_return_df, simos_adjclose_df = get_simos_data(return_df, adjclose_df) # simos period, only trading days

## for filtering
def get_tradables(adjclose_df, trading_date=subconfig.PORTFOLIO_DATE):
    sid_list = adjclose_df.columns

    notnull = adjclose_df.loc[trading_date, :].notnull()
    notzero = adjclose_df.loc[trading_date, :] != 0

    return sid_list[notnull * notzero]

def is_tradables(sid_list, tradables):
    tradables = set(tradables)

    return np.array([True if sid in tradables else False for sid in sid_list])

def get_daconsids(krx_df):
    krx_df.columns = ['date', 'code', 'name', 'volume', 'open', 'high', 'low', 'close']
    dacon_sid_list = [ii[1:] for ii in krx_df['code'].unique()] # 060310 형식으로 바꿔줌

    return dacon_sid_list

def is_daconsids(sid_list, daconsids):
    daconsids = set(daconsids)

    return np.array([True if sid in daconsids else False for sid in sid_list])

class Submission:
    holding_return_s = (simos_adjclose_df.loc[SIMOS_END, :] - simos_adjclose_df.loc[SIMOS_START, :]).divide(simos_adjclose_df.loc[SIMOS_START, :])  
    holding_return_s = holding_return_s.fillna(0)

    # simos_winners = 
    # TODO: Add data science evaluation metrics

    # TODO: Make not-instance-specific variables to class variables
    def __init__(self, alpha_series:pd.Series, alpha_name:str, top=200, bottom=200):
        self.alpha_series = alpha_series
        self.alpha_name = alpha_name
        self.top = top
        self.bottom = bottom

        self.sid_list = self.alpha_series.index
        self.tradables = get_tradables(adjclose_df)
        self.daconsids = get_daconsids(krx_df)
    
        self.is_selectables = is_tradables(self.sid_list, self.tradables) * is_daconsids(self.sid_list, self.daconsids)
        self.submission_df = None
        self.alpha_winners = None
        self.alpha_losers = None

        # for excess return
        self.long_hpr = None
        self.short_hpr = None
        self.final_return = None

        # for variance
        self.long_returns = None
        self.short_returns = None
        
    def get_rank(self, export_path=None):
        selectables = self.alpha_series[self.is_selectables]
        top_s = selectables.nlargest(self.top)
        bottom_s = selectables.nsmallest(self.bottom)
        
        self.alpha_winners = top_s.index
        self.alpha_losers = bottom_s.index
        
        submission_df = pd.DataFrame(
            data={'rank': [-1]*DACON_SID_CNT},
            index=self.daconsids
        )
        submission_df.index.name = 'sid'

        submission_df['rank'][top_s.index] = np.arange(1, self.top+1)
        submission_df['rank'][bottom_s.index] = np.arange(DACON_SID_CNT, DACON_SID_CNT - self.bottom, -1)

        submission_df['rank'][submission_df['rank'] == -1] = np.arange(self.top+1, DACON_SID_CNT - self.bottom + 1)

        self.submission_df = submission_df

        if export_path:
            submission_df.index = ['A' + idx for idx in submission_df.index]
            submission_df.index.name = '종목코드'
            submission_df.columns = ['순위']
            submission_df.to_csv(export_path / f'{self.alpha_name}.csv', encoding='utf-8')
            
            print(f'Saved to {export_path / self.alpha_name}.csv')
            return submission_df

        return submission_df

    def get_excess_return(self, risk_free_rate=0.035, days_of_trading=15):
        self.long_hpr = Submission.holding_return_s[self.alpha_winners].sum()
        self.short_hpr = Submission.holding_return_s[self.alpha_losers].sum()

        self.final_return = (self.long_hpr - self.short_hpr) / 400

        annualized_final_return = self.final_return * 250 / days_of_trading
        excess_return = annualized_final_return - risk_free_rate

        return excess_return
    
    def get_volatility(self, days_of_trading=15):
        self.long_returns = simos_return_df.loc[:, self.alpha_winners].mean(axis=1)
        self.short_returns = simos_return_df.loc[:, self.alpha_losers].mean(axis=1)

        annualized_portfolio_returns = (self.long_returns - self.short_returns) / 2 * 250
        annualized_mean_returns = annualized_portfolio_returns.mean()
        
        annualized_portfolio_volatility = np.sqrt((annualized_portfolio_returns - annualized_mean_returns).pow(2)[2:].sum() / (days_of_trading-2))

        return annualized_portfolio_volatility

    def get_Sharpe(self):
        return self.get_excess_return() / self.get_volatility()

    
class Score:
    holding_return_s = (simos_adjclose_df.loc[SIMOS_END, :] - simos_adjclose_df.loc[SIMOS_START, :]).divide(simos_adjclose_df.loc[SIMOS_START, :])  
    holding_return_s = holding_return_s.fillna(0)

    def __init__(self, submission_csv_filepath, alpha_name, top=200, bottom=200, encoding='utf-8'):
        self.alpha_name = alpha_name
        self.top = top
        self.bottom = bottom

        with open(submission_csv_filepath, 'r', encoding=encoding) as f:
            submission_df = pd.read_csv(f, index_col=0)
        
        submission_df.index = [idx[1:] for idx in submission_df.index]
        submission_df.index.name = 'sid'
        submission_df.columns = ['rank']

        self.alpha_series = submission_df['rank']
        self.sid_list = self.alpha_series.index

        # TODO: Add validations

        self.submission_df = None
        self.alpha_winners = self.alpha_series.nsmallest(self.top).index
        self.alpha_losers = self.alpha_series.nlargest(self.bottom).index

        # for excess return
        self.long_hpr = None
        self.short_hpr = None
        self.final_return = None

        # for variance
        self.long_returns = None
        self.short_returns = None
    
    def get_excess_return(self, risk_free_rate=0.035, days_of_trading=15):
        self.long_hpr = Score.holding_return_s[self.alpha_winners].sum()
        self.short_hpr = Score.holding_return_s[self.alpha_losers].sum()

        self.final_return = (self.long_hpr - self.short_hpr) / 400

        annualized_final_return = self.final_return * 250 / days_of_trading
        excess_return = annualized_final_return - risk_free_rate

        return excess_return

    def get_volatility(self, days_of_trading=15):
        self.long_returns = simos_return_df.loc[:, self.alpha_winners].mean(axis=1)
        self.short_returns = simos_return_df.loc[:, self.alpha_losers].mean(axis=1)

        annualized_portfolio_returns = (self.long_returns - self.short_returns) / 2 * 250
        annualized_mean_returns = annualized_portfolio_returns.mean()
        
        annualized_portfolio_volatility = np.sqrt((annualized_portfolio_returns - annualized_mean_returns).pow(2)[2:].sum() / (days_of_trading-2))

        return annualized_portfolio_volatility

    def get_Sharpe(self):
        sharpe = self.get_excess_return() / self.get_volatility()
        print(f'Sharpe of {self.alpha_name}: {sharpe}')

        return sharpe